# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
import tensorflow as tf
import pandas as pd
import keras
from keras.datasets import cifar10
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from sklearn.feature_extraction.text import CountVectorizer

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 31s 0us/step


In [3]:
y_test

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]])

In [0]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(50000):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])

In [0]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:

Y1_train =  keras.utils.to_categorical(np.asarray(Y1_train), 5)
Y1_test =  keras.utils.to_categorical(np.asarray(Y1_test), 5)


Y2_train =  keras.utils.to_categorical(np.asarray(Y2_train), 10)
Y2_test =  keras.utils.to_categorical(np.asarray(Y2_test), 10)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [15]:
model = Sequential()

model.add(Conv2D(32, 5, 5, input_shape=(32, 32, 3), activation='relu',name='conv1'))
model.add(Conv2D(16, 5, 5, activation='relu',name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(32, 32, 3..., activation="relu", name="conv1")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), activation="relu", name="conv2")`
  after removing the cwd from sys.path.


In [13]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

hist1 = model.fit(x1_train, y1_train,
         nb_epoch=20,
         shuffle=True,
         batch_size=50,
         validation_data=(x1_val, y1_val), verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
 - 2s - loss: 0.5199 - acc: 0.8008 - val_loss: 0.7128 - val_acc: 0.7416
Epoch 2/20
 - 2s - loss: 0.5112 - acc: 0.8015 - val_loss: 0.7706 - val_acc: 0.7222
Epoch 3/20
 - 2s - loss: 0.4836 - acc: 0.8155 - val_loss: 0.7576 - val_acc: 0.7324
Epoch 4/20
 - 2s - loss: 0.4865 - acc: 0.8124 - val_loss: 0.7545 - val_acc: 0.7316
Epoch 5/20
 - 2s - loss: 0.4769 - acc: 0.8170 - val_loss: 0.7624 - val_acc: 0.7288
Epoch 6/20
 - 3s - loss: 0.4580 - acc: 0.8259 - val_loss: 0.7395 - val_acc: 0.7342
Epoch 7/20
 - 3s - loss: 0.4441 - acc: 0.8303 - val_loss: 0.7710 - val_acc: 0.7394
Epoch 8/20
 - 3s - loss: 0.4510 - acc: 0.8278 - val_loss: 0.7828 - val_acc: 0.7358
Epoch 9/20
 - 2s - loss: 0.4280 - acc: 0.8364 - val_loss: 0.8619 - val_acc: 0.7118
Epoch 10/20
 - 2s - loss: 0.4173 - acc: 0.8433 - val_loss: 0.7734 - val_acc: 0.7410
Epoch 11/20
 - 2s - loss: 0.4172 - acc: 0.8396 - val_loss: 0.7842 - val_acc: 0.7348
Epoch 12/20
 - 2s - loss: 0.4141 - a

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for layer in model.layers:
  if('conv'  in layer.name): #prefix detection to freeze layers which does not have dense
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [20]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist2 = trans_model.fit(x2_train, y2_train, nb_epoch=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2304)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               295040    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 2s - loss: 1.1630 - acc: 0.5336 - val_loss: 0.9725 - val_acc: 0.6248
Epoch 2/10
 - 1s - loss: 0.9929 - acc: 0.6145 - val_loss: 0.9026 - val_acc: 0.6548
Epoch 3/10
 - 1s - loss: 0.9555 - acc: 0.6274 - val_loss: 0.8720 - val_acc: 0.6672
Epoch 4/10
 - 1s - loss: 0.9200 - acc: 0.6486 - val_loss: 0.8428 - val_acc: 0.6862
Epoch 5/10
 - 1s - loss: 0.8924 - acc: 0.6590 - val_loss: 0.8354 - val_acc: 0.6830
Epoch 6/10
 - 1s - loss: 0.8675 - acc: 0.6684 - val_loss: 0.8168 - val_acc: 0.6886
Epoch 7/10
 - 1s - loss: 0.8499 - acc: 0.6735 - val_loss: 0.7795 - val_acc: 0.7068
Epoch 8/10
 - 1s - loss: 0.8280 - acc: 0.6831 - val_loss: 0.7583 - val_acc: 0.7158
Epoch 9/10
 - 1s - loss: 0.8187 - acc: 0.6901 - val_loss: 0.7675 - val_acc: 0.7078
Epoch 10/10
 - 1s - loss: 0.8026 - acc: 0.6951 - val_loss: 0.7772 - val_acc: 0.7024


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [23]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [25]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [26]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [27]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
cvec = CountVectorizer()
train_data_features=cvec.fit(twenty_train.data)
cvecdata= cvec.transform(twenty_train.data)

In [0]:
df=pd.DataFrame(cvecdata.todense(),columns=cvec.get_feature_names())

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [36]:
from sklearn.linear_model import LogisticRegression
y=twenty_train.target
model=LogisticRegression()
log_model=model.fit(df,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [38]:
X_test=cvec.transform(twenty_test.data)
y_test=twenty_test.target
log_model.score(X_test,y_test)

0.9014647137150466